In [ ]:
import io

import numpy as np
import torch
from chitra.core import load_imagenet_labels
from chitra.serve.cloud.aws_serverless import ChaliceServer
from smart_open import open as smart_open
from timm import create_model, list_models

In [ ]:
def load_model(file):
    return (lambda x: x + 1)

In [ ]:
import boto3

s3 = boto3.client('s3')

In [ ]:
response = s3.list_buckets()

In [ ]:
# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

In [ ]:
file = s3.download_file('gflow-models', 'efficientnet_b0.pth', 'model.pth')

In [ ]:
BUCKET = 'gflow-models'
FILE_OBJ = 'efficientnet_b0.pth'
MODEL_PATH = "s3://gflow-models/efficientnet_b0.pth"

In [ ]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(BUCKET)
file_object = bucket.Object(FILE_OBJ)

In [ ]:
buffer = io.BytesIO()
file_object.download_fileobj(buffer)

In [ ]:
model = create_model('efficientnet_b0', pretrained=False)

with smart_open(MODEL_PATH, 'rb') as f:
    buffer = io.BytesIO(f.read())
    model = model.load_state_dict(torch.load(buffer))

In [ ]:
# server = ChaliceServer('image-classification',
#                        "s3://gflow-models/efficientnet_b0.pth",
#                        model_loader=load_model)

In [ ]:
from chitra.image import Chitra

In [ ]:
LABELS = load_imagenet_labels()

In [ ]:
model = create_model(
    'efficientnet_b0',
    pretrained=True,
    checkpoint_path='./efficientnet_b0_ra-3dd342df.pth').eval()

In [ ]:
image = Chitra(
    "https://ichef.bbci.co.uk/news/976/cpsprodpb/67CF/production/_108857562_mediaitem108857561.jpg"
)
image.resize((256, 256))

In [ ]:
x = image.numpy().astype(np.float32)
x = x / 255.
x = torch.from_numpy(x)

LABELS[model(x.permute(2, 0, 1).unsqueeze(0)).argmax(1)]